In [1]:
import derm_ita as dita

import os
import io
from io import BytesIO
import numpy as np
import pandas as pd

from PIL import Image
import cv2
import h5py
import tensorflow
from tensorflow.keras.utils import load_img, img_to_array, save_img

In [2]:
CWD = os.getcwd()

In [3]:
def ITA_FST(image):
    img_ita = dita.get_cropped_center_ita(image)
    groh_FST = None
    if 23 <= img_ita:
        groh_FST = 12
    elif 0 <= img_ita < 23:
        groh_FST = 34
    elif img_ita < 0:
        groh_FST = 56
    
    return groh_FST

In [4]:
benign_imgs = []
malignant_imgs = []

In [5]:
for img in os.listdir(f'{CWD}/benign'):
    if img == '.DS_Store':
        continue
    else:
        benign_imgs.append(img)

for img in os.listdir(f'{CWD}/malignant'):
    if img == '.DS_Store':
        continue
    else:
        malignant_imgs.append(img)

In [6]:
n_imgs = len(benign_imgs) + len(malignant_imgs)
print(n_imgs)

1898


In [7]:
images = np.empty(shape=[n_imgs, 64, 64, 3])
ids = []
labels = []
malignances = np.empty(shape=[n_imgs])
skin_tones = np.empty(shape=[n_imgs])

In [8]:
idx = 0
for img in benign_imgs:
    byteImgIO = io.BytesIO()
    byteImg = Image.open(f'{CWD}/benign/{img}')
    byteImg.save(byteImgIO, "JPEG")

    image = load_img(f'{CWD}/benign/{img}', target_size=(64,64))
    id = img
    label = 'benign'
    malignance = 0
    skin_tone = ITA_FST(image)

    image = img_to_array(image, dtype=np.uint8)
    assert image.shape == (64, 64, 3)
    assert image.dtype == np.uint8
    images[idx] = image

    images[idx] = image
    ids.append(id)
    labels.append(label)
    malignances[idx] = malignance
    skin_tones[idx] = skin_tone

    save_img(f'{CWD}/all_images/{img}', image)
    idx += 1

In [9]:
for img in malignant_imgs:
    byteImgIO = io.BytesIO()
    byteImg = Image.open(f'{CWD}/malignant/{img}')
    byteImg.save(byteImgIO, "JPEG")

    image = load_img(f'{CWD}/malignant/{img}', target_size=(64,64))
    id = img
    label = 'malignant'
    malignance = 1
    skin_tone = ITA_FST(image)

    image = img_to_array(image, dtype=np.uint8)
    assert image.shape == (64, 64, 3)
    assert image.dtype == np.uint8
    images[idx] = image

    images[idx] = image
    ids.append(id)
    labels.append(label)
    malignances[idx] = malignance
    skin_tones[idx] = skin_tone

    save_img(f'{CWD}/all_images/{img}', image)
    idx += 1

/Users/tyleryang/.pyenv/versions/3.10.14/envs/tf-gpu/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/tyleryang/.pyenv/versions/3.10.14/envs/tf-gpu/lib/python3.10/site-packages/derm_ita/ita_core_computations.py:75: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(l)
/Users/tyleryang/.pyenv/versions/3.10.14/envs/tf-gpu/lib/python3.10/site-packages/derm_ita/ita_core_computations.py:86: RuntimeWarning: Mean of empty slice
  ita = math.atan2(np.nanmean(l) - 50, np.nanmean(b)) * (180 / np.pi)


In [10]:
assert len(ids) == idx == n_imgs == len(labels) == len(malignances) == len(skin_tones)

metadata = {
    'id': ids,
    'label': labels,
    'malignance': malignances,
    'skin_tone': skin_tones,
}
metadata = pd.DataFrame(metadata)
metadata.to_excel(f'{CWD}/metadata_BrAD.xlsx')